# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Capstone'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

dataset = Dataset.get_by_name(ws, name='covid-19-measures-cases-weekly')
df = dataset.to_pandas_dataframe()
train_data = df.iloc[:-49]
test_data = df.iloc[-50:]

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "worker"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [22]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(time_column_name='week', 
                                               forecast_horizon=10,
                                               time_series_id_column_names=["country"],
                                               freq='W',
                                               target_lags='auto',
                                               target_rolling_window_size=10)

automl_config = AutoMLConfig(task='forecasting',                             
                             primary_metric='normalized_root_mean_squared_error',
                             blocked_models = ['ExtremeRandomTrees', 'AutoArima', 'Prophet'],       
                             experiment_timeout_hours=0.3,
                             training_data=dataset,
                             label_column_name='rate_14_day',
                             compute_target=compute_target,
                             enable_early_stopping=False,
                             n_cross_validations=30,                             
                             verbosity=logging.INFO,
                             forecasting_parameters=forecasting_parameters)                                                             

In [28]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [29]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [30]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="Covid-19-aftermath-3",
    workspace=ws,    
    steps=[automl_step])

In [31]:
remote_run = experiment.submit(pipeline)

Created step automl_module [71a44c37][57056e7b-1a93-4e36-8840-e92af472a768], (This step will run and generate new outputs)
Submitted PipelineRun dd13962c-4b1f-46a2-82bb-d5f242a022ac
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Capstone/runs/dd13962c-4b1f-46a2-82bb-d5f242a022ac?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/capstone


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [32]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [33]:
remote_run.wait_for_completion()

PipelineRunId: dd13962c-4b1f-46a2-82bb-d5f242a022ac
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Capstone/runs/dd13962c-4b1f-46a2-82bb-d5f242a022ac?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/capstone
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 9213a29f-a51b-415d-ba54-7b54b2318e94
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Capstone/runs/9213a29f-a51b-415d-ba54-7b54b2318e94?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/capstone
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '9213a29f-a51b-415d-ba54-7b54b2318e94', 'target': 'worker', 'status': 'Completed', 'startTimeUtc': '2021-02-05T13:00:38.992536Z', 'endTimeUtc': '2021-02-05T13:30:34.734711Z', 'properties': {'ContentSnapshotId': 'ce106ddb-1f1

'Finished'

In [34]:
metrics_output = remote_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

Downloaded azureml/9213a29f-a51b-415d-ba54-7b54b2318e94/metrics_data, 1 files out of an estimated total of 1


,9213a29f-a51b-415d-ba54-7b54b2318e94_6,9213a29f-a51b-415d-ba54-7b54b2318e94_8,9213a29f-a51b-415d-ba54-7b54b2318e94_10,9213a29f-a51b-415d-ba54-7b54b2318e94_9,9213a29f-a51b-415d-ba54-7b54b2318e94_7,9213a29f-a51b-415d-ba54-7b54b2318e94_11
normalized_root_mean_squared_error,[0.4680279827952882],[0.4490204899805414],[0.4790149774442213],[0.43521980962558265],[0.4408959680415582],[0.4349341379582462]
mean_absolute_error,[152.3679943631077],[148.61930025733452],[150.47646846367346],[147.29828646461573],[150.26607197951472],[144.165063046842]
normalized_root_mean_squared_log_error,[0.2728431635645],[0.27966411656853574],[0.26554625064888987],[0.273284203885111],[0.2842519823684439],[0.2570696576809009]
median_absolute_error,[93.89215155429046],[87.8283681156991],[91.86853709882207],[87.59991070890315],[88.93284321669617],[83.96516516111504]
root_mean_squared_error,[232.60081278546295],[230.0817093148482],[230.42321317911075],[228.75078242438607],[232.883090495313],[224.86961026692944]
normalized_median_absolute_error,[0.36258641696222066],[0.3343667381640037],[0.3758935795910562],[0.3244624058547206],[0.32389650886928484],[0.3302735319105047]
spearman_correlation,[0.21030703526716932],[0.21478605599010808],[0.23202390957736024],[0.2577974297593894],[0.23272735955162924],[0.2755307169096398]
root_mean_squared_log_error,[1.4735435542337256],[1.599578416237949],[1.387665583074417],[1.5780451878831039],[1.6379344374493587],[1.3655980367955074]
explained_variance,[-0.3137022895856728],[-0.4801404788458008],[-0.41827105291255806],[-0.521645565742935],[-0.6721372591744349],[-0.3143578724325526]
mean_absolute_percentage_error,[363.5327745745606],[346.2007318330028],[399.433113842998],[301.85693396947005],[325.40890969545217],[315.2093480431469]


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [35]:
# Retrieve best model from Pipeline Run
best_model_output = remote_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/9213a29f-a51b-415d-ba54-7b54b2318e94/model_data, 1 files out of an estimated total of 1


In [37]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(featurization_config=None,
                                                                           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
                                                    ('prefittedsoftvotingregressor',
                                                     PreFittedSoftVotingRegressor(estimators=[('9',
                                                                                               Pipeline(memory=None,
                                                                                                        steps=[('robustscaler',
                                                                                                                RobustScaler(copy=True,
                                                                       

In [38]:
# part of last data per country
forecast_context = train_data[train_data.country=='Netherlands'][-5:]
# repeat the last row 12 times
test_set = pd.DataFrame(np.repeat(forecast_context[-1:].values, 12, axis=0))
test_set.columns = forecast_context.columns

# make a daterange in the future
forecast_horizon = pd.date_range(pd.Timestamp(2021, 2, 4), pd.Timestamp(2021, 4, 30), freq="W")

X_forecast = test_set.reset_index()
X_forecast['week'] = forecast_horizon
X_forecast['index'] = forecast_horizon
X_forecast = X_forecast.set_index('index')

# test_labels = test_set.pop('rate_14_day').values
# forecast_context.pop('rate_14_day').values


In [46]:
forecast_horizon

DatetimeIndex(['2021-02-07', '2021-02-14', '2021-02-21', '2021-02-28',
               '2021-03-07', '2021-03-14', '2021-03-21', '2021-03-28',
               '2021-04-04', '2021-04-11', '2021-04-18', '2021-04-25'],
              dtype='datetime64[ns]', freq='W-SUN')

In [45]:
test_labels = X_forecast['rate_14_day'].values
test_labels

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
      dtype=object)

In [41]:
# add the forecast context to the horizon
# X_forecast = forecast_context.append(X_forecast)
test_labels = X_forecast['rate_14_day'].values
test_labels.fill(np.NaN)

label_fcst, data_trans = best_model.forecast(X_forecast, y_pred=test_labels,ignore_data_errors=True)


In [200]:

label_fcst, data_trans = best_model.forecast(forecast_destination=pd.Timestamp(2021, 3, 4))

In [310]:
label_fcst

array([111.54117142, 111.54117142, 111.54117142, 111.54117142,
       111.54117142, 111.54117142, 111.54117142, 111.54117142,
        94.47158431,  94.47158431,  79.19267178,  79.19267178])

In [307]:
# data_trans[['_automl_target_col']]
data_trans[0]

KeyError: 0

In [229]:
best_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(featurization_config=None,
                                                                           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
                                                    ('prefittedsoftvotingregressor',
                                                     PreFittedSoftVotingRegressor(estimators=[('7',
                                                                                               Pipeline(memory=None,
                                                                                                        steps=[('robustscaler',
                                                                                                                RobustScaler(copy=True,
                                                                       

In [42]:
remote_run.upload_file("outputs/my_model.pickle", best_model_output._path_on_datastore)
automl_model = remote_run.register_model(model_path="outputs/my_model.pickle", model_name='Covid-19-cases-forecaster-2', description='Forecast the number of cases per 100.000 of infections with covid-19.')
print(automl_model.name, automl_model.version, sep = '\t')


Covid-19-cases-forecaster-2	1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [43]:
model_run = next(next(remote_run.get_children()).get_children())

In [263]:
model_run.download_files('outputs', 'outputs')

In [44]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name = "covid-19", file_path = "outputs/outputs/conda_env_v_1_0_0.yml")
inference_config = InferenceConfig(entry_script='outputs/outputs/scoring_file_v_1_0_3.py', environment=myenv)

In [47]:
from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1,
    auth_enabled = True, 
    enable_app_insights = True, 
    tags = {'category':'training', 'trait':'ML'})

# deploy_configuration(cpu_cores=None, memory_gb=None, tags=None, properties=None, description=None, location=None, auth_enabled=None, ssl_enabled=None, enable_app_insights=None, ssl_cert_pem_file=None, ssl_key_pem_file=None, ssl_cname=None, dns_name_label=None, primary_key=None, secondary_key=None, collect_model_data=None, cmk_vault_base_url=None, cmk_key_name=None, cmk_key_version=None, vnet_name=None, subnet_name=None)

In [48]:
from azureml.core.model import Model
service = Model.deploy(ws, "covid-19-cases-forecaster-2", [automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [50]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = ''
# If the service is authenticated, set the key or token
key = ''

# Two sets of data to score, so we get two results back
data = {"data":
    [
        {
            "country": "Netherlands",
            "week": "2021-02-07T00:00:00",
            "AdaptationOfWorkplace": "false",
            "AdaptationOfWorkplacePartial": "false",
            "BanOnAllEvents": "false",
            "BanOnAllEventsPartial": "false",
            "ClosDaycare": "false",
            "ClosDaycarePartial": "false",
            "ClosHigh": "false",
            "ClosHighPartial": "false",
            "ClosPrim": "false",
            "ClosPrimPartial": "false",
            "ClosPubAny": "false",
            "ClosPubAnyPartial": "false",
            "ClosSec": "false",
            "ClosSecPartial": "false",
            "ClosureOfPublicTransport": "false",
            "ClosureOfPublicTransportPartial": "false",
            "EntertainmentVenues": "false",
            "EntertainmentVenuesPartial": "false",
            "GymsSportsCentres": "false",
            "GymsSportsCentresPartial": "false",
            "HotelsOtherAccommodation": "false",
            "HotelsOtherAccommodationPartial": "false",
            "IndoorOver100": "false",
            "IndoorOver1000": "false",
            "IndoorOver50": "false",
            "IndoorOver500": "false",
            "MasksMandatoryAllSpaces": "false",
            "MasksMandatoryAllSpacesPartial": "false",
            "MasksMandatoryClosedSpaces": "false",
            "MasksMandatoryClosedSpacesPartial": "false",
            "MasksVoluntaryAllSpaces": "false",
            "MasksVoluntaryAllSpacesPartial": "false",
            "MasksVoluntaryClosedSpaces": "false",
            "MasksVoluntaryClosedSpacesPartial": "false",
            "MassGather50": "false",
            "MassGather50Partial": "false",
            "MassGatherAll": "false",
            "MassGatherAllPartial": "false",
            "NonEssentialShops": "false",
            "NonEssentialShopsPartial": "false",
            "OutdoorOver100": "false",
            "OutdoorOver1000": "false",
            "OutdoorOver50": "false",
            "OutdoorOver500": "false",
            "PlaceOfWorship": "false",
            "PlaceOfWorshipPartial": "false",
            "PrivateGatheringRestrictions": "false",
            "PrivateGatheringRestrictionsPartial": "false",
            "RegionalStayHomeOrder": "false",
            "RegionalStayHomeOrderPartial": "false",
            "RestaurantsCafes": "false",
            "RestaurantsCafesPartial": "false",
            "SocialCircle": "false",
            "SocialCirclePartial": "false",
            "StayHomeGen": "false",
            "StayHomeGenPartial": "false",
            "StayHomeOrder": "false",
            "StayHomeOrderPartial": "false",
            "StayHomeRiskG": "false",
            "StayHomeRiskGPartial": "false",
            "Teleworking": "false",
            "TeleworkingPartial": "false",
            "WorkplaceClosures": "false",
            "WorkplaceClosuresPartial": "false"
        },
        {
            "country": "Netherlands",
            "week": "2021-02-14T00:00:00",
            "AdaptationOfWorkplace": "false",
            "AdaptationOfWorkplacePartial": "false",
            "BanOnAllEvents": "false",
            "BanOnAllEventsPartial": "false",
            "ClosDaycare": "false",
            "ClosDaycarePartial": "false",
            "ClosHigh": "false",
            "ClosHighPartial": "false",
            "ClosPrim": "false",
            "ClosPrimPartial": "false",
            "ClosPubAny": "false",
            "ClosPubAnyPartial": "false",
            "ClosSec": "false",
            "ClosSecPartial": "false",
            "ClosureOfPublicTransport": "false",
            "ClosureOfPublicTransportPartial": "false",
            "EntertainmentVenues": "false",
            "EntertainmentVenuesPartial": "false",
            "GymsSportsCentres": "false",
            "GymsSportsCentresPartial": "false",
            "HotelsOtherAccommodation": "false",
            "HotelsOtherAccommodationPartial": "false",
            "IndoorOver100": "false",
            "IndoorOver1000": "false",
            "IndoorOver50": "false",
            "IndoorOver500": "false",
            "MasksMandatoryAllSpaces": "false",
            "MasksMandatoryAllSpacesPartial": "false",
            "MasksMandatoryClosedSpaces": "false",
            "MasksMandatoryClosedSpacesPartial": "false",
            "MasksVoluntaryAllSpaces": "false",
            "MasksVoluntaryAllSpacesPartial": "false",
            "MasksVoluntaryClosedSpaces": "false",
            "MasksVoluntaryClosedSpacesPartial": "false",
            "MassGather50": "false",
            "MassGather50Partial": "false",
            "MassGatherAll": "false",
            "MassGatherAllPartial": "false",
            "NonEssentialShops": "false",
            "NonEssentialShopsPartial": "false",
            "OutdoorOver100": "false",
            "OutdoorOver1000": "false",
            "OutdoorOver50": "false",
            "OutdoorOver500": "false",
            "PlaceOfWorship": "false",
            "PlaceOfWorshipPartial": "false",
            "PrivateGatheringRestrictions": "false",
            "PrivateGatheringRestrictionsPartial": "false",
            "RegionalStayHomeOrder": "false",
            "RegionalStayHomeOrderPartial": "false",
            "RestaurantsCafes": "false",
            "RestaurantsCafesPartial": "false",
            "SocialCircle": "false",
            "SocialCirclePartial": "false",
            "StayHomeGen": "false",
            "StayHomeGenPartial": "false",
            "StayHomeOrder": "false",
            "StayHomeOrderPartial": "false",
            "StayHomeRiskG": "false",
            "StayHomeRiskGPartial": "false",
            "Teleworking": "false",
            "TeleworkingPartial": "false",
            "WorkplaceClosures": "false",
            "WorkplaceClosuresPartial": "false"
        }
    ]
}
input_data = json.dumps(data)
service.run(input_data)

'{"forecast": [516.0555766729951, 518.8032188317503], "index": [{"week": 1612656000000, "country": "Netherlands", "origin": 1612051200000}, {"week": 1613260800000, "country": "Netherlands", "origin": 1612051200000}]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [51]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-02-05T13:37:01,525690100+00:00 - gunicorn/run 
2021-02-05T13:37:01,551440100+00:00 - rsyslog/run 
2021-02-05T13:37:01,549068700+00:00 - nginx/run 
2021-02-05T13:37:01,573898300+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [303]:
service.delete()